# Instructor

 Using instructor & pydantic to control LLM output

In [165]:

import instructor
from pydantic import BaseModel, Field
from typing import Iterable, Literal
from typing import List, Optional, Union
from graphviz import Digraph
from dotenv import load_dotenv
import enum
import os
from openai import OpenAI
from openai import OpenAIError
import json
import pandas as pd
from langchain_core.prompts import PromptTemplate
from pprint import pprint
from pathlib import Path
from utils import *

# Load the environment variables
from dotenv import load_dotenv
load_dotenv()


In [22]:
# load data set
data = json.load(open('../data/ConvFinQA/data/train.json'))
df = pd.DataFrame(data)

usecols = ['q', 'q_type', 'q_difficulty', 'q_chain', 'a_chain', 'answer_0', 'exe_ans_0', 'answer_1', 
           'exe_ans_1','pre_text', 'post_text',  'table_ori', 'table', 'annotation']

df['q_type'] = df['id'].map(lambda x: 'Type I' if 'Single' in x  else 'Type II')  # get question type
df['q_difficulty'] = df.annotation.map(get_difficulty)                            # get question difficulty
df['q'] = df.apply(get_q, axis=1)                                                 # get top level question
df['q_chain'] = df.apply(lambda x: x['annotation']['dialogue_break'], axis=1)     # get question chain
df['a_chain'] = df.apply(lambda x: x['annotation']['exe_ans_list'], axis=1)       # get answer chain

df['answers'] = df.apply(get_a, axis=1)                                               # get answer
df['answer_0'] = df.answers.map(lambda x: x.get('answer_0').strip())     # answer
df['exe_ans_0'] = df.answers.map(lambda x: x.get('exe_ans_0'))     # execution answer
df['answer_1'] = df.answers.map(lambda x: x.get('answer_1', '').strip())     # answer
df['exe_ans_1'] = df.answers.map(lambda x: x.get('exe_ans_1'))     # execution answer

df[usecols].head()

,q,q_type,q_difficulty,q_chain,a_chain,answer_0,exe_ans_0,answer_1,exe_ans_1,pre_text,post_text,table_ori,table,annotation
0,{'question_0': 'what was the percentage change...,Type I,{0: 4},[what is the net cash from operating activitie...,"[206588.0, 181001.0, 25587.0, 0.14136]",14.1%,0.14136,,None,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...","[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'amt_table': '<table class='wikitable'><tr><t...
1,{'question_0': 'what was the percent of the gr...,Type I,{0: 4},"[what were revenues in 2008?, what were they i...","[9362.2, 9244.9, 117.3, 0.01269]",1.3%,0.01269,,None,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'amt_table': '<table class='wikitable'><tr><t...
2,{'question_0': 'what was the percentage change...,Type I,{0: 4},"[what was the total of net sales in 2001?, and...","[5363.0, 7983.0, -2620.0, -0.3282]",-32%,-0.3282,,None,[in a new business model such as the retail se...,[.],"[[, 2002, 2001, 2000], [Net sales, $5,742, $5,...","[[, 2002, 2001, 2000], [net sales, $ 5742, $ 5...",{'amt_table': '<table class='wikitable'><tr><t...
3,{'question_0': 'what was the difference in per...,Type I,{0: 6},[what was the change in the performance of the...,"[-24.05, -0.2405, 102.11, 2.11, 0.0211, -0.2616]",-26.16%,-0.2616,,None,[( 1 ) includes shares repurchased through our...,[.],"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",{'amt_table': '<table class='wikitable'><tr><t...
4,{'question_0': 'what is the roi of an investme...,Type II,"{0: 2, 1: 5}",[what was the fluctuation of the performance p...,"[-8.94, -0.0894, -24.05, -0.2405, 2.11, 0.0211...",-8.9%,-0.0894,-26.16%,-0.2616,[( 1 ) includes shares repurchased through our...,[.],"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",{'amt_table': '<table class='wikitable'><tr><t...


In [79]:
# prompts
system_prompt = """I am a highly intelligent bot. I can have conversations with the user to
answer a series of questions. Later questions may depend on previous questions to answer. You need
to provide me with the series of questions as the context and I will answer the last question.
"""

prompt_template = PromptTemplate.from_template(
    "Answer the following questions using the context provided:\ncontext: {context} \nquestions:\n{questions} \nanswer: {answer}"
)


In [80]:
for i, row in df.head(1).iterrows():
    print(f'row {i}: {row.q}')
    
    # prompt
    context = row.annotation.get('amt_table') + '\n ' + row.annotation.get('amt_pre_text') + '\n '+ row.annotation.get('amt_post_text')
    questions = '\n'.join([f'Q{j+1}: {s}' for j,s in enumerate(row.annotation['dialogue_break'])])
    prompt = prompt_template.format(context=context, questions=questions, answer="")
    print(prompt)

row 0: {'question_0': 'what was the percentage change in the net cash from operating activities from 2008 to 2009'}
Answer the following questions using the context provided:
context: <table class='wikitable'><tr><td>1</td><td>2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009</td></tr><tr><td>2</td><td>net income</td><td>$ 103102</td><td>$ 104222</td><td>$ 104681</td></tr><tr><td>3</td><td>non-cash expenses</td><td>74397</td><td>70420</td><td>56348</td></tr><tr><td>4</td><td>change in receivables</td><td>21214</td><td>-2913 ( 2913 )</td><td>-28853 ( 28853 )</td></tr><tr><td>5</td><td>change in deferred revenue</td><td>21943</td><td>5100</td><td>24576</td></tr><tr><td>6</td><td>change in other assets and liabilities</td><td>-14068 ( 14068 )</td><td>4172</td><td>17495</td></tr><tr><td>7</td><td>net cash from operating activities</td><td>$ 206588</td><td>$ 181001</td><td>$ 174247</td></tr></table>
 26 | 2009 annual report in fis

### Create base model for multiple hop QA

In [ ]:
client = instructor.from_openai(OpenAI())

In [ ]:
class AmountType(str, enum.Enum):
    """Enumeration representing the types of numerical amounts that can be used in an answer."""
    PERCENT = "PERCENT"
    NET_AMOUNT = "NET_AMOUNT"
    OTHER = "OTHER"


class OperationType(str, enum.Enum):
    """Enumeration representing the types of operations that can be used in an answer."""
    ADD = "ADD"
    SUBTRACT = "SUBTRACT"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"
    EXP = "EXP"
    GREATER = "GREATER"
    ASK_FOR_NUMBER = "ASK_FOR_NUMBER"


class Operation(BaseModel):
    """Class representing a single operation in an answer list"""

    operation: str = Field(
        ...,
        description=("Operation used to calculate the answer for example: add, multiply, subtract, divide, exp, greater. format as operation(arg1, arg2). "
                     "examples: add(10, 20), ask_for_number(arg1), divide(2000, 120)"
                     "Leave percentage calculations as decimals ie do not multiply by 100"
                     "if the operation is a query on the text to retrieve an amount then format as: Ask for number arg1"
                    
                ),
    )
    operation_type: OperationType = Field(
        description="Operation used to calculate the answer for example: add, multiply, subtract, divide, exp, greater.",
    )

    arg1: str = Field(
        description="the first number used by the operation",
    )
    arg2: Optional[str] = Field(
        description="the second number used by the operation",
    )

    def __str__(self):
        return f"{self.operation_type}({self.arg1}, {self.arg2})"


class Answer(BaseModel):
    """Class representing a single answer in an answer list"""

    id: int = Field(..., description="Unique id of the answer")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    explanation: str = Field(
        ...,
        description="Explanation of the answer and the calculation",
    )
    amount: float = Field(
        ...,
        description="Amount of the answer",
    )
    operation: Operation = Field(
        ...,
        description=("Operation used to calculate the answer for example: add, multiply, subtract, divide, exp, greater. format as operation(arg1, arg2). "
                     "for example: add(10, 20). "
                     "Leave percentage calculations as decimals ie do not multiply by 100"
                     "if the operation is a query on the text to retrieve an amount then format as: Ask for number arg1"
                    
                ),
    )
    dependencies: List[int] = Field(
        default_factory=list,
        description="List of ids of any previous answers that need to be calculated before this answer can be calculated",
    )
    amount_type: AmountType = Field(
        ...,
        description="Type of amount asked for the question, either a net amount or a percent or other",
    )


class AnswerList(BaseModel):
    """Container class representing a tree of questions to ask a question answering system."""

    answer_list: List[Answer] = Field(
        ..., description="The list of answers"
    )

    def _dependencies(self, ids: List[int]) -> List[Answer]:
        """Returns the dependencies of a query given their ids."""
        return [q.id for q in self.answer_list if q.id in ids]



In [168]:

def structuredQA(system_prompt: str, user_prompt:str, model = "gpt-4o-mini") -> AnswerList:
    """ 
    Format the response from the model into a structured answer list using the custom pydanctic model AnswerList
    
    args:
    system_prompt: str: system prompt
    user_prompt: str: user prompt
    model: str: model to use for the completion
    return:
    AnswerList: list of answers
    """

    messages = [ { "role": "system", "content": system_prompt, }, 
                 { "role": "user", "content": user_prompt, }  ]
    answers_list = client.chat.completions.create(
        model=model,
        temperature=0,
        response_model=AnswerList,
        messages=messages,
        max_tokens=1000,
    )
    return answers_list

In [ ]:

answers = structuredQA(system_prompt, prompt)
for a in answers.answer_list:
    print(a.id)
    print(a.question)
    print(a.explanation)
    print(a.amount)
    print(a.operation)
    print(a.dependencies)
    print(a.amount_type)
    print('---')

In [123]:
df.annotation[0]

{'amt_table': "<table class='wikitable'><tr><td>1</td><td>2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009</td></tr><tr><td>2</td><td>net income</td><td>$ 103102</td><td>$ 104222</td><td>$ 104681</td></tr><tr><td>3</td><td>non-cash expenses</td><td>74397</td><td>70420</td><td>56348</td></tr><tr><td>4</td><td>change in receivables</td><td>21214</td><td>-2913 ( 2913 )</td><td>-28853 ( 28853 )</td></tr><tr><td>5</td><td>change in deferred revenue</td><td>21943</td><td>5100</td><td>24576</td></tr><tr><td>6</td><td>change in other assets and liabilities</td><td>-14068 ( 14068 )</td><td>4172</td><td>17495</td></tr><tr><td>7</td><td>net cash from operating activities</td><td>$ 206588</td><td>$ 181001</td><td>$ 174247</td></tr></table>",
 'amt_pre_text': '26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within 

In [ ]:
model = "gpt-4o-mini"
res = []    # save working results

for i, row in df.iterrows():
    print(f'row {i}: {row.q}')
    
    # prompt
    context = row.annotation.get('amt_table') + '\n ' + row.annotation.get('amt_pre_text') + '\n '+ row.annotation.get('amt_post_text')
    questions = '\n'.join([f'Q{j+1}: {s}' for j,s in enumerate(row.annotation['dialogue_break'])])
    prompt = prompt_template.format(context=context, questions=questions, answer="")
    print(prompt)
    # call the API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "assistant",
        "content": [
            {
            "type": "text",
            "text": assistant_role
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": prompt
            }
        ]
        }, 
    ],
    # todo
    temperature=0.5,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    response_text = response.choices[0].message.content

## Planning and executing a query plan
https://python.useinstructor.com/examples/planning-tasks/

In [25]:

class QueryType(str, enum.Enum):
    """Enumeration representing the types of queries that can be asked to a question answer system."""

    SINGLE_QUESTION = "SINGLE"
    MERGE_MULTIPLE_RESPONSES = "MERGE_MULTIPLE_RESPONSES"


class Query(BaseModel):
    """Class representing a single question in a query plan."""

    id: int = Field(..., description="Unique id of the query")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    dependencies: List[int] = Field(
        default_factory=list,
        description="List of sub questions that need to be answered before asking this question",
    )
    node_type: QueryType = Field(
        default=QueryType.SINGLE_QUESTION,
        description="Type of question, either a single question or a multi-question merge",
    )


class AnsweringPlan(BaseModel):
    """Container class representing a tree of questions to ask a question answering system."""

    query_graph: List[Query] = Field(
        ..., description="The query graph representing the plan"
    )

    def _dependencies(self, ids: List[int]) -> List[Query]:
        """Returns the dependencies of a query given their ids."""
        return [q for q in self.query_graph if q.id in ids]


In [42]:
system_prompt = ("You are a world class query planning algorithm capable of breaking apart questions into its dependency queries such that the answers can be used to inform the parent question." 
"Do not answer the questions, simply provide a correct compute graph with good specific questions to ask and relevant dependencies. "
"when asked to calculate a percentage amount, you should first ask the user for the total amount and then the percentage amount. "
"Before you call the function, think step-by-step to get a better understanding of the problem." )


In [43]:
system_prompt

'You are a world class query planning algorithm capable ofbreaking apart questions into its dependency queries such that the answers can be used to inform the parent question.Do not answer the questions, simply provide a correct compute graph with good specific questions to ask and relevant dependencies. when asked to calculate a percentage amount, you should first ask the user for the total amount and then the percentage amount. Before you call the function, think step-by-step to get a better understanding of the problem.'

In [44]:
# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.from_openai(OpenAI())


def structuredQA(question: str) -> AnsweringPlan:
    PLANNING_MODEL = "gpt-4-0613"

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"Consider: {question}\nGenerate the correct query plan.",
        },
    ]

    root = client.chat.completions.create(
        model=PLANNING_MODEL,
        temperature=0,
        response_model=AnsweringPlan,
        messages=messages,
        max_tokens=1000,
    )
    return root


In [51]:
for i, q in temp[['q_', 'q_chain']].head(2).values:
    print(i)
    for q_item in q:
        print(f'..{q_item}')
    print('query_planner:')
    qp = structuredQA(i)
    for i in qp.query_graph:
        print(i)
    
    print('---\n\n')

what was the percentage change in the net cash from operating activities from 2008 to 2009
..what is the net cash from operating activities in 2009?
..what about in 2008?
..what is the difference?
..what percentage change does this represent?
query_planner:
id=1 question='What was the net cash from operating activities in 2008?' dependencies=[] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=2 question='What was the net cash from operating activities in 2009?' dependencies=[] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=3 question='What is the difference between the net cash from operating activities in 2009 and 2008?' dependencies=[1, 2] node_type=<QueryType.MERGE_MULTIPLE_RESPONSES: 'MERGE_MULTIPLE_RESPONSES'>
id=4 question='What is the percentage change in the net cash from operating activities from 2008 to 2009?' dependencies=[1, 3] node_type=<QueryType.MERGE_MULTIPLE_RESPONSES: 'MERGE_MULTIPLE_RESPONSES'>
---


what was the percent of the growth in the revenues from 2007 

In [54]:
from graphviz import Digraph
# plot the graph using graphviz
dot = Digraph(comment="Query Plan", node_attr={"shape": "plaintext"})

for query in qp.query_graph:
    dot.node(str(query.id), query.question)

    for dep_id in query.dependencies:
        dot.edge(str(dep_id), str(query.id))

# Render the graph
dot.render("knowledge_graph.gv", view=True)

'knowledge_graph.gv.pdf'

In [28]:
# plan = query_planner(q)
# plan.model_dump()


{'query_graph': [{'id': 1,
   'question': 'What was the net cash from operating activities in 2008?',
   'dependencies': [],
   'node_type': <QueryType.SINGLE_QUESTION: 'SINGLE'>},
  {'id': 2,
   'question': 'What was the net cash from operating activities in 2009?',
   'dependencies': [],
   'node_type': <QueryType.SINGLE_QUESTION: 'SINGLE'>},
  {'id': 3,
   'question': 'What was the percentage change in the net cash from operating activities from 2008 to 2009?',
   'dependencies': [1, 2],
   'node_type': <QueryType.MERGE_MULTIPLE_RESPONSES: 'MERGE_MULTIPLE_RESPONSES'>}]}

## Knowledge Graph
https://python.useinstructor.com/examples/knowledge_graph/#generating-knowledge-graphs

In [11]:
class Node(BaseModel):
    id: int
    label: str
    color: str


class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str = "black"


class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)

In [16]:


# Adds response_model to ChatCompletion
# Allows the return of Pydantic model rather than raw JSON
client = instructor.from_openai(OpenAI())


def generate_graph(input) -> KnowledgeGraph:
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"Help me understand the following by describing it as a detailed knowledge graph: {input}",
            }
        ],
        response_model=KnowledgeGraph,
    )  # type: ignore

In [17]:
def visualize_knowledge_graph(kg: KnowledgeGraph):
    dot = Digraph(comment="Knowledge Graph")

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    dot.render("knowledge_graph.gv", view=True)

graph = generate_graph("Teach me about quantum mechanics")
visualize_knowledge_graph(graph)